In [550]:
import re
import json
from numpy import random

from cswEngine import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [551]:
path,RFC = Exp(RFC_bag=get_RFC_bag('subject and victim')).gen_path()


In [553]:
path[0].get_filled_state(RFC)

"One morning, Olivia walked into Joe's coffee shop."

for n in path:
  if n.type == 'story_node':
    print(n.get_filled_state(RFC))
    


In [545]:
# filled_state = self.state
# filled_state = sent
role_property_L = re.findall("\[.*?\]",filled_state)
for rp in role_property_L:
  r,p = rp[1:-1].split('.')
  filled_state = re.sub("\\"+rp,RFC[r][p],filled_state)


"Adam splashed a coffee into Silvia's face."

In [539]:
sent = "[subject.name] splashed a [drink1.name] into [victim.name]'s face."

re.sub("\[subject.name\]","Silvia",sent,1)
# filled_state

"Silvia splashed a [drink1.name] into [victim.name]'s face."

In [129]:
schema_role_fillers_D = read_json('fight.roles')['schema_role_fillers']
filler_properties_D = read_json('fight.roles')['filler_properties']

In [280]:
filler_properties_D

{'coffee': {'name': 'coffee'},
 'dessert1': {'name': 'cookie'},
 'dessert2': {'name': 'cake'},
 'person1': {'emotion': 'rage',
  'name': 'Bill',
  'type': 'nurse',
  'violent': 'True',
  'yell': 'I hate coffee!'},
 'person2': {'emotion': 'rage2',
  'name': 'Silvia',
  'type': 'doctor',
  'violent': 'True',
  'yell': 'I hate coffee2!'},
 'person3': {'name': 'Adam', 'violent': 'True'},
 'person4': {'name': 'Olivia', 'violent': 'False'},
 'setting1': {'name': "Joe's coffee shop"},
 'tea1': {'name': 'green tea'},
 'tea2': {'name': 'black tea'}}

In [159]:
schema_role_fillers_D

{'dessert': ['dessert1', 'dessert2'],
 'drink1': ['tea1', 'tea2', 'coffee'],
 'drink2': ['tea1', 'tea2', 'coffee'],
 'drink3': ['tea1', 'tea2', 'coffee'],
 'setting': ['setting1'],
 'subject': ['person3', 'person4'],
 'victim': ['person1', 'person2']}

In [491]:
sent = "[subject.name] splashed a [drink1.name] into [victim.name]'s face."
role_property_L = re.findall("\[.*?\]",sent,0)

# for r,p in list(map(lambda x: x[1:-1].split('.'), role_property_L)):
for rp in role_property_L:
  print(rp)


[subject.name]
[drink1.name]
[victim.name]


In [284]:
def make_RFC_bag_full(schema_role_fillers_D,filler_properties_D):
  """ makes every possible RFC
  role_filler_str_L: [["role1-filler1","role1-filler2"],["role2-filler1","role2-filler2"]...]
  all_role_fillers: [("role1-filler1","role2-filler1"),("role1-filler2","role2-filler2")]
  RFC_str: {'role':'filler',}
  which I can pass to get_filler_properties, 
  """
  # 1 make_role_filler_str_L
  role_filler_str_L = []
  for role,filler_L in schema_role_fillers_D.items():
    role_L = []
    for filler in filler_L:
      role_filler_str = "%s-%s"%(role,filler)
      role_L.append(role_filler_str)
    role_filler_str_L.append(role_L)
  # 2 all_role_filers
  all_RFC_tups = list(itertools.product(*role_filler_str_L))
  # 3 RFC_str
  RFC_bag = []
  for RFC_str_tup in all_RFC_tups:
    RFC_str = {rf_str.split('-')[0]:rf_str.split('-')[1] for rf_str in RFC_str_tup}
    RFC = get_filler_properties(RFC_str,filler_properties_D) 
    RFC_bag.append(RFC)
  return RFC_bag


RFC_bag_full = make_RFC_bag_full(schema_role_fillers_D,filler_properties_D)

one aggresive subject, one non aggressive subject, two victims
Olivia Adam Bill Silvia

In [419]:
for i,j in set(itertools.permutations([0,0,1,1],2)):
  print(i,j)

0
1
0
1


In [440]:
RFC_bag = []
vict_names = ['Bill','Silvia']
subj_names = ['Adam','Olivia']
random.shuffle(subj_names)

for i in [0,1]:
  for RFC in RFC_bag_full:
    if RFC['subject']['name'] == subj_names[0]: 
        if RFC['victim']['name'] == vict_names[i]:
          RFC_bag.append(RFC)
          break
        
for RFC in RFC_bag:
  print(RFC['victim']['name'],RFC['subject']['name'])

Bill Olivia
Silvia Olivia


In [344]:
def get_poor_filled_bag(RFC_bag_full):
  random.shuffle(RFC_bag_full)
  RFC_bag = []
  names_L = ['Olivia','Adam','Bill','Silvia']
  random.shuffle(names_L)
  for name in names_L:
    for RFC in RFC_bag_full:
      if RFC['subject']['name'] == name:
        RFC_bag.append(RFC)
        print(name)
        break
  return RFC_bag
get_poor_filled_bag(RFC_bag_full)

Adam
Olivia


[{'dessert': {'name': 'cake'},
  'drink1': {'name': 'green tea'},
  'drink2': {'name': 'green tea'},
  'drink3': {'name': 'green tea'},
  'setting': {'name': "Joe's coffee shop"},
  'subject': {'name': 'Adam', 'violent': 'True'},
  'victim': {'emotion': 'rage',
   'name': 'Bill',
   'type': 'nurse',
   'violent': 'True',
   'yell': 'I hate coffee!'}},
 {'dessert': {'name': 'cake'},
  'drink1': {'name': 'green tea'},
  'drink2': {'name': 'black tea'},
  'drink3': {'name': 'coffee'},
  'setting': {'name': "Joe's coffee shop"},
  'subject': {'name': 'Olivia', 'violent': 'False'},
  'victim': {'emotion': 'rage2',
   'name': 'Silvia',
   'type': 'doctor',
   'violent': 'True',
   'yell': 'I hate coffee2!'}}]